<a href="https://colab.research.google.com/github/AbhirKarande/OCRandProductRecognition/blob/main/MetaLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# Set the directories containing the image samples
data_dir = "<path_to_directory_containing_sample_images>"
class_names = ["class1", "class2", "class3", "class4"]
num_classes = len(class_names)
num_samples_per_class = 5
num_support_samples = 3  # Number of support samples per class
num_query_samples = 2  # Number of query samples per class

In [ ]:
# Create empty lists for storing support set and query set
x_support = []
y_support = []
x_query = []
y_query = []

In [ ]:
# Load the image samples and assign labels
for i, class_name in enumerate(class_names):
    class_dir = os.path.join(data_dir, class_name)
    for j in range(num_samples_per_class):
        img_path = os.path.join(class_dir, f"{class_name}_{j+1}.jpg")
        img = load_img(img_path, target_size=(224, 224))  # Adjust target_size as needed
        img_array = img_to_array(img)
        
        if j < num_support_samples:
            x_support.append(img_array)
            y_support.append(i)
        else:
            x_query.append(img_array)
            y_query.append(i)


In [ ]:
# Convert lists to numpy arrays
x_support = np.array(x_support)
y_support = np.array(y_support)
x_query = np.array(x_query)
y_query = np.array(y_query)

# Normalize pixel values to [0, 1]
x_support = x_support / 255.0
x_query = x_query / 255.0

In [ ]:
# Define the Prototypical Networks model
def prototypical_networks(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

In [ ]:
# Define the loss function
def prototypical_loss(y_true, y_pred):
    support_labels = y_true[:, :num_support_samples]
    query_labels = y_true[:, num_support_samples:]
    support_predictions = y_pred[:, :num_support_samples, :]
    query_predictions = y_pred[:, num_support_samples:, :]
    
    # Compute prototypes by taking the mean of support set embeddings
    prototypes = keras.backend.mean(support_predictions, axis=1)
    
    # Calculate distances between query embeddings and prototypes
    distances = keras.backend.sum(keras.backend.square(query_predictions - prototypes), axis=-1)
    
    # Compute cross-entropy loss
    query_loss = keras.backend.sparse_categorical_crossentropy(query_labels, -distances)
    
    return keras.backend.mean(query_loss)

In [ ]:
# Create the Prototypical Networks model
input_shape = x_support.shape[1:]
model = prototypical_networks(input_shape)

In [ ]:
model.compile(optimizer=Adam(), loss=prototypical_loss)


In [ ]:
model.fit(x_support, y_support, epochs=10, batch_size=num_classes)


In [ ]:
# Evaluate the model on query set
_, accuracy = model.evaluate(x_query, y_query)
print("Accuracy:", accuracy)